In [3]:
import librosa
import numpy as np
import os
import soundfile as sf
import pickle


In [4]:

def load_audio(file_path, sr=16000, mono=True):
    """
    Loads an audio file, resamples it to the desired sample rate, and converts to mono if needed.
    """
    audio, _ = librosa.load(file_path, sr=sr, mono=mono)
    return audio

def split_into_clips(audio, clip_length_sec=10, sr=16000):
    """
    Splits audio into fixed-length clips. Pads the last clip if it's shorter than the desired length.
    """
    clip_length = clip_length_sec * sr  # Length of each clip in samples
    num_clips = int(np.ceil(len(audio) / clip_length))  # Number of clips
    clips = []

    for i in range(num_clips):
        start = i * clip_length
        end = start + clip_length
        clip = audio[start:end]
        
        # If the last clip is shorter than the clip length, pad it with zeros
        if len(clip) < clip_length:
            clip = np.pad(clip, (0, clip_length - len(clip)), 'constant')

        clips.append(clip)
    
    return np.array(clips)

def normalize_audio(audio):
    """
    Normalizes the audio to the range [-1, 1].
    """
    return audio / np.max(np.abs(audio))

def audio_to_spectrogram(audio, n_fft=1024, hop_length=512):
    stft = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
    spectrogram = np.abs(stft)  # Magnitude spectrogram
    return spectrogram

def remove_invalid_values(audio):
    """
    Removes non-finite values (NaN, inf) from the audio by replacing them with zero.
    """
    # Replace NaN or inf values with 0
    audio = np.nan_to_num(audio, nan=0.0, posinf=0.0, neginf=0.0)
    return audio


In [5]:
noise_dir = r"C:\Users\Siddhartha Devan V\Downloads\Fellowship\201-360\201-360\Noised"
clean_dir = r"C:\Users\Siddhartha Devan V\Downloads\Fellowship\201-360\201-360\Denoised"

In [7]:
noise_audio_files = os.listdir(noise_dir)
clean_audio_files = os.listdir(clean_dir)

# noise_audio_files = [os.path.join(noise_dir, i) for i in noise_audio_files]
# clean_audio_files = [os.path.join(clean_dir, i) for i in clean_audio_files]

print(noise_audio_files[:5])
print(clean_audio_files[:5])

# noise_audio_files = noise_audio_files[:3]
# clean_audio_files = clean_audio_files[:3]


['201.wav', '202.wav', '203.wav', '204.wav', '205.wav']
['201.wav', '202.wav', '203.wav', '204.wav', '205.wav']


In [8]:
target_noise_dir_audio = r"C:\Users\Siddhartha Devan V\Downloads\Fellowship\201-360\201-360\N1"
target_clean_dir_audio = r"C:\Users\Siddhartha Devan V\Downloads\Fellowship\201-360\201-360\DN1"

target_noise_dir_spect = r"C:\Users\Siddhartha Devan V\Downloads\Fellowship\201-360\201-360\Noised_spectogram"
target_clean_dir_spect = r"C:\Users\Siddhartha Devan V\Downloads\Fellowship\201-360\201-360\Denoised_Spectogram"

def audio_splitter(noise_dir, clean_dir, noise_audio_files, clean_audio_files, target_noise_dir_audio, target_clean_dir_audio, target_noise_dir_spect, target_clean_dir_spect):

    for i, j in zip(noise_audio_files, clean_audio_files):
        if i == j:
            noisy_audio = load_audio(os.path.join(noise_dir, i), sr=16000, mono=True)
            clean_audio = load_audio(os.path.join(clean_dir, j), sr=16000, mono=True)

            noisy_clips = split_into_clips(noisy_audio, clip_length_sec=10, sr=16000)
            clean_clips = split_into_clips(clean_audio, clip_length_sec=10, sr=16000)
             
            noisy_clips = [normalize_audio(clip) for clip in noisy_clips]
            clean_clips = [normalize_audio(clip) for clip in clean_clips]

            noisy_clips = [remove_invalid_values(clip) for clip in noisy_clips]
            clean_clips = [remove_invalid_values(clip) for clip in clean_clips]

            noisy_spectrograms = [audio_to_spectrogram(clip) for clip in noisy_clips]
            clean_spectrograms = [audio_to_spectrogram(clip) for clip in clean_clips]

            for clip_num in range(len(noisy_clips)):
                noisy_clip = noisy_clips[clip_num]
                clean_clip = clean_clips[clip_num]
                noisy_spectrogram = noisy_spectrograms[clip_num]
                clean_spectrogram = clean_spectrograms[clip_num]

                sf.write(os.path.join(target_noise_dir_audio, i[:-4]+'_'+str(clip_num)+'.wav'), noisy_clip, 16000)  # Using librosa to save
                sf.write(os.path.join(target_clean_dir_audio, j[:-4]+'_'+str(clip_num)+'.wav'), clean_clip, 16000) 
                
                np.save(os.path.join(target_noise_dir_spect, i[:-4]+'_'+str(clip_num)), noisy_spectrogram)
                np.save(os.path.join(target_clean_dir_spect, j[:-4]+'_'+str(clip_num)), clean_spectrogram)

                print(i, 'done saving the audio and spectrograms')

            
audio_splitter(noise_dir, clean_dir, noise_audio_files, clean_audio_files, target_noise_dir_audio, target_clean_dir_audio, target_noise_dir_spect, target_clean_dir_spect)
    






201.wav done saving the audio and spectrograms
202.wav done saving the audio and spectrograms
203.wav done saving the audio and spectrograms
204.wav done saving the audio and spectrograms
205.wav done saving the audio and spectrograms
206.wav done saving the audio and spectrograms
207.wav done saving the audio and spectrograms
208.wav done saving the audio and spectrograms
209.wav done saving the audio and spectrograms
210.wav done saving the audio and spectrograms
211.wav done saving the audio and spectrograms
212.wav done saving the audio and spectrograms
213.wav done saving the audio and spectrograms
214.wav done saving the audio and spectrograms
215.wav done saving the audio and spectrograms
216.wav done saving the audio and spectrograms
217.wav done saving the audio and spectrograms
218.wav done saving the audio and spectrograms
219.wav done saving the audio and spectrograms
220.wav done saving the audio and spectrograms
221.wav done saving the audio and spectrograms
222.wav done 

In [ ]:
import os
import librosa
import numpy as np
import pickle
import soundfile as sf

def save_stft(audio, sr, n_fft=1024, hop_length=512):
    """
    Compute and return the STFT of the audio.
    """
    stft = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
    spectrogram = np.abs(stft)  # Magnitude spectrogram
    return spectrogram

def split_save_audio_and_stft(audio, sr, clip_length, raw_save_dir, stft_save_dir, base_filename, start_index, n_fft=1024, hop_length=512):
    """
    Split the audio into 10-second clips, save the raw audio, and compute & save the STFT.
    """
    num_clips = len(audio) // clip_length
    for i in range(num_clips):
        # Get 10-second audio clip
        clip = audio[i * clip_length: (i + 1) * clip_length]
        
        # Save raw audio
        raw_filename = f"{base_filename}_clip_{start_index + i}.wav"
        sf.write(os.path.join(raw_save_dir, raw_filename), clip, sr)
        
        # Compute and save STFT
        stft = save_stft(clip, sr, n_fft, hop_length)
        stft_filename = f"{base_filename}_clip_{start_index + i}.pkl"
        with open(os.path.join(stft_save_dir, stft_filename), 'wb') as f:
            pickle.dump(stft, f)

def preprocess_save_audio_and_stft(noisy_dir, clean_dir, output_noisy_raw_dir, output_noisy_stft_dir, output_clean_raw_dir, output_clean_stft_dir, sr=16000, clip_length_sec=10, n_fft=1024, hop_length=512):
    """
    Preprocess and split the audio into 10-second clips, save both raw audio and STFT spectrograms.
    """
    noisy_files = sorted(os.listdir(noisy_dir))
    clean_files = sorted(os.listdir(clean_dir))
    
    clip_length = clip_length_sec * sr  # Length of each 10-second clip in samples

    os.makedirs(output_noisy_raw_dir, exist_ok=True)
    os.makedirs(output_noisy_stft_dir, exist_ok=True)
    os.makedirs(output_clean_raw_dir, exist_ok=True)
    os.makedirs(output_clean_stft_dir, exist_ok=True)

    for idx, (noisy_file, clean_file) in enumerate(zip(noisy_files, clean_files)):
        # Load audio files
        noisy_audio, _ = librosa.load(os.path.join(noisy_dir, noisy_file), sr=sr)
        clean_audio, _ = librosa.load(os.path.join(clean_dir, clean_file), sr=sr)
        
        # Split and save noisy raw audio and STFT
        base_filename_noisy = os.path.splitext(noisy_file)[0]
        split_save_audio_and_stft(noisy_audio, sr, clip_length, output_noisy_raw_dir, output_noisy_stft_dir, base_filename_noisy, idx * 100, n_fft, hop_length)
        
        # Split and save clean raw audio and STFT
        base_filename_clean = os.path.splitext(clean_file)[0]
        split_save_audio_and_stft(clean_audio, sr, clip_length, output_clean_raw_dir, output_clean_stft_dir, base_filename_clean, idx * 100, n_fft, hop_length)

# Paths to input and output folders
noisy_dir = 'path_to_noisy_audio'
clean_dir = 'path_to_clean_audio'
output_noisy_raw_dir = 'path_to_output_noisy_raw_clips'
output_noisy_stft_dir = 'path_to_output_noisy_stft_clips'
output_clean_raw_dir = 'path_to_output_clean_raw_clips'
output_clean_stft_dir = 'path_to_output_clean_stft_clips'

# Preprocess and save both raw audio and STFT spectrograms
preprocess_save_audio_and_stft(noisy_dir, clean_dir, output_noisy_raw_dir, output_noisy_stft_dir, output_clean_raw_dir, output_clean_stft_dir)
